<a href="https://colab.research.google.com/github/nmcardoso/galmorpho/blob/master/splus_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x 
import random
import tensorflow as tf
import numpy as np
import os
import glob
import multiprocessing as mp
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from keras import backend as K
from keras import layers
from keras import models
from keras import optimizers
from keras import utils
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import VGG16
from progressbar import progressbar

# !pip3 install git+https://github.com/nmcardoso/fitsbook-python
# import fitsbook as fb

In [0]:
# Reprodutibilidade
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

os.environ['PYTHONHASHSEED'] = '12'
np.random.seed(123)
random.seed(1234)
tf.set_random_seed(12345)

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, 
#                               inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

print('Sementes Plantadas!')

Sementes Plantadas!


## Preparação dos dados

In [0]:
!cp "/content/drive/My Drive/datasets/stamps_scaled_12ch_subset_64px.tar" dataset.tar

In [0]:
!cp /content/drive/'My Drive'/datasets/stamps_scaled_12ch_subset_64px.csv /content/dataset.csv

In [0]:
!mkdir dataset

In [0]:
!tar -C dataset -xvf dataset.tar

In [0]:
class NpySequence(utils.Sequence):
  """Generate data for Keras"""
  def __init__(self, files, labels, classes, image_size=(256, 256), channels=3, batch_size=32, output_channels=3, channel_range=None):
    self.files = files
    self.labels = self.__to_categorical(labels, classes)
    self.image_size = image_size
    self.channels = channels
    self.output_channels = output_channels
    self.channel_range = channel_range
    self.batch_size = batch_size
    self.indexes = [i for i in range(len(files))]
    np.random.shuffle(self.indexes)
  
  def __len__(self):
    """The number of batches per epoch"""
    return int(np.floor(len(self.files) / self.batch_size))
  
  def __getitem__(self, index):
    """Generate one batch of data"""
    partition = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    X, Y = self.__data_generation(partition)
    if self.channel_range:
      return X[..., self.channel_range[0]:self.channel_range[1]], Y
    elif self.channels > self.output_channels:
      return [X[..., 0:3], X[..., 3:6], X[..., 6:9], X[..., 9:12]], Y
    else:
      return X, Y
  
  def on_epoch_end(self):
    """Shuffle files list and remap labels"""
    np.random.shuffle(self.indexes)
    
  def __to_categorical(self, labels, classes):
    """Labels to numbers"""
    classes_dict = {c:i for i,c in enumerate(classes)}
    return [classes_dict[l] for l in labels]

  def __data_generation(self, partition):
    """Load npy file and associate with a label"""
    X = np.empty((self.batch_size, *self.image_size, self.channels))
    Y = np.empty((self.batch_size,), dtype=int)

    for i, index in enumerate(partition):
      X[i,] = np.load(self.files[index])
      Y[i] = self.labels[index]

    return X, Y

## Modelos

### Não usados

In [0]:
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
  # 1x1 conv
  conv1 = layers.Conv2D(f1, (1, 1), padding='same', activation='relu')(layer_in)

  # 3x3 conv
  conv3 = layers.Conv2D(f2_in, (1, 1), padding='same', activation='relu')(layer_in)
  conv3 = layers.Conv2D(f2_out, (3, 3), padding='same', activation='relu')(layer_in)

  # 5x5 conv
  conv5 = layers.Conv2D(f3_in, (1, 1), padding='same', activation='relu')(layer_in)
  conv5 = layers.Conv2D(f3_out, (5, 5), padding='same', activation='relu')(conv5)

  # 3x3 max pooling
  pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(layer_in)
  pool = layers.Conv2D(f4_out, (1, 1), padding='same', activation='relu')(pool)

  # concatenate filters, assumes filters/channels last
  layer_out = layers.concatenate([conv1, conv3, conv5, pool], axis=-1)
  return layer_out

def compile_inception_model(image_size):
  input_layer = layers.Input(shape=image_size + (3,))
  conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
  pool1 = layers.MaxPool2D((2, 2))(conv1)
  conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
  pool2 = layers.MaxPool2D((2, 2))(conv2)
  iblock1 = inception_module(input_layer, 32, 48, 64, 8, 16, 16)
  iblock2 = inception_module(iblock1, 64, 48, 96, 16, 48, 32)
  flatten = layers.Flatten()(iblock2)
  dense1 = layers.Dense(1024, activation='relu')(flatten)
  dropout1 = layers.Dropout(.3)(dense1)
  dense2 = layers.Dense(512, activation='relu')(dropout1)
  dropout2 = layers.Dropout(.3)(dense2)
  dense3 = layers.Dense(256, activation='relu')(dropout2)
  dropout3 = layers.Dropout(.3)(dense3)
  output = layers.Dense(1, activation='sigmoid')(dropout3)
  model = models.Model(inputs=input_layer, outputs=output)
  model.name = 'splus_inception'
  utils.plot_model(model, to_file='model.png')
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

def compile_inception_pretrained(image_size):
  base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(.5)(x)
  x = layers.Dense(256, activation='relu')(x)
  x = layers.Dropout(.5)(x)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = models.Model(inputs=base_model.input, outputs=output)
  model.name = 'splus_inception_pretrained'

  for layer in base_model.layers:
    layer.trainable = False
  
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  
  return model

def compile_VGG16(image_size):
  model = models.Sequential()
  model.name = 'splus_vgg16_impl'
  model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(image_size + (3,))))
  model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(2048, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

def compile_multidim_model(image_size):
  input_tensor = layers.Input(shape=image_size + (1,))

  branch_outputs = []
  for i in range(3):
    out = layers.Lambda(lambda x: x[:, i])(input_tensor)
    out = layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(image_size + (1,)))(out)
    out = layers.MaxPooling2D((2, 2))(out)
    out = layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_size + (1,)))(out)
    out = layers.MaxPooling2D((2, 2))(out)
    out = layers.Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(image_size + (1,)))(out)
    out = layers.MaxPooling2D((2, 2))(out)
    branch_outputs.append(out)
  
  x = layers.Concatenate()(branch_outputs)
  x = layers.Flatten()(x)
  x = layers.Dense(1024, activation='relu')(x)
  x = layers.Dropout(.5)(x)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(.5)(x)
  x = layers.Dense(256, activation='relu')(x)
  x = layers.Dropout(.5)(x)
  x = layers.Dense(1, activation='sigmoid')(x)

  model = models.Model(inputs=input_tensor, outputs=x)
  model.name = 'splus_multidim_conv'
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  utils.plot_model(model, to_file='model.png')
  return model

### Usados

In [0]:
def compile_parallel_model(image_size, channels=3):
  # inception_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
  # for layer in inception_model.layers:
  #     layer.trainable = False

  # input_layer = layers.Input(shape=image_size + (channels,))
  branch_outputs = []
  inputs = []
  for i in range(int(channels / 3)):
    # x = layers.Lambda(lambda t: t[..., (i * 3):(i * 3 + 2)])(input_layer)
    # base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_tensor=x)
    # for layer in base_model.layers:
    #   layer.trainable = False
    # j = inception_model.output
    # x = layers.GlobalAveragePooling2D()(j)
    input_layer = layers.Input(shape=image_size + (3,), name=f'InputImage_{i + 1}')
    inputs.append(input_layer)
    x = layers.Conv2D(32, (3, 3), activation='relu', name=f'Conv2D_{i + 1}1')(input_layer)
    x = layers.MaxPooling2D((2, 2), name=f'MaxPooling2D_{i + 1}1')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', name=f'Conv2D_{i + 1}2')(x)
    x = layers.MaxPooling2D((2, 2), name=f'MaxPooling2D_{i + 1}2')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', name=f'Conv2D_{i + 1}3')(x)
    x = layers.MaxPooling2D((2, 2), name=f'MaxPooling2D_{i + 1}3')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', name=f'Conv2D_{i + 1}4')(x)
    x = layers.MaxPooling2D((2, 2), name=f'MaxPooling2D_{i + 1}4')(x)
    # x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    # x = layers.MaxPooling2D((2, 2))(x)
    branch_outputs.append(x)
  x = layers.Concatenate(name='Mixer')(branch_outputs)
  x = layers.Flatten(name='Flatten')(x)
  # x = layers.Dense(2048, activation='relu')(x)
  # x = layers.Dropout(.5)(x)
  x = layers.Dense(2048, activation='relu', name='Dense_1')(x)
  x = layers.Dropout(.5, name='Dropout_1')(x)
  x = layers.Dense(1024, activation='relu', name='Dense_2')(x)
  x = layers.Dropout(.5, name='Dropout_2')(x)
  x = layers.Dense(1024, activation='relu', name='Dense_3')(x)
  x = layers.Dropout(.5, name='Dropout_3')(x)
  x = layers.Dense(512, activation='relu', name='Dense_4')(x)
  x = layers.Dropout(.5, name='Dropout_4')(x)
  x = layers.Dense(1, activation='sigmoid', name='Dense_5')(x)
  model = models.Model(inputs=inputs, outputs=x)
  model.name = 'multi_input_64'
  model.compile(optimizer=optimizers.RMSprop(lr=6e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  utils.plot_model(model, to_file='model.png')
  return model
compile_parallel_model((64, 64), 12)

In [0]:
def preprocess(channel_range=None):
  global train_generator, test_generator, val_generator
  df = pd.read_csv('/content/dataset.csv')
  E = df[df.CLASS == 'E']
  S = df[df.CLASS == 'S']

  # train_df = pd.concat([E[0:3127], S[0:951]])
  train_df = pd.concat([E[0:1400], S[0:1000]])
  train_X = [f'/content/dataset/{f}.npy' for f in train_df.ID.tolist()]
  train_Y = train_df.CLASS.tolist()

  # val_df = pd.concat([E[3128:5004], S[952:1522]])
  val_df = pd.concat([E[1401:2100], S[1001:1500]])
  val_X = [f'/content/dataset/{f}.npy' for f in val_df.ID.tolist()]
  val_Y = val_df.CLASS.tolist()

  # test_df = pd.concat([E[5005:], S[1523:]])
  test_df = pd.concat([E[2101:], S[1501:]])
  test_X = [f'/content/dataset/{f}.npy' for f in test_df.ID.tolist()]
  test_Y = test_df.CLASS.tolist()

  train_generator = NpySequence(train_X, train_Y, ['E', 'S'], image_size=(64, 64), channels=12, batch_size=20, output_channels=3)
  val_generator = NpySequence(val_X, val_Y, ['E', 'S'], image_size=(64, 64), channels=12, batch_size=20, output_channels=3)
  test_generator = NpySequence(test_X, test_Y, ['E', 'S'], image_size=(64, 64), channels=12, batch_size=20, output_channels=3)

def compile_model(image_size):
  model = models.Sequential()
  model.name = 'splus_convolutional'
  model.add(layers.Conv2D(1024, (3, 3), activation='relu', input_shape=(image_size + (12,))))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(512, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(2048, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizers.RMSprop(lr=5e-4),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

def fit(model, epochs=1, workers=1):
  global history, train_generator, val_generator
  use_mp = workers > 1
  history = model.fit_generator(
      train_generator,
      # steps_per_epoch=80,
      # validation_steps=30,
      epochs=epochs,
      validation_data=val_generator,
      callbacks=[fb.callbacks.FitsbookCallback()],
      use_multiprocessing=use_mp,
      workers=workers
  )

## Treinamento

In [0]:
preprocess()
m = compile_parallel_model((64, 64), 12)
fit(m, 300)